## 新词挖掘

---

### 基于频次的新词挖掘

In [2]:
import jieba
text = "支持向量机是一类按监督学习方式对数据进行二元分类的广义线性分类器，其决策边界是对学习样本求解的最大边距超平面。\
支持向量机使用铰链损失函数计算经验风险并在求解系统中加入了正则化项以优化结构风险，是一个具有稀疏性和稳健性的分类器。\
铰链损失函数的思想就是让那些未能正确分类的和正确分类的之间的距离要足够的远。\
支持向量机可以通过核方法进行非线性分类，是常见的核学习方法之一。\
支持向量机被提出于1964年，在二十世纪90年代后得到快速发展并衍生出一系列改进和扩展算法，\
在人像识别、文本分类等模式识别问题中有得到应用。"
words = jieba.lcut(text)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\DELL\AppData\Local\Temp\jieba.cache
Loading model cost 0.552 seconds.
Prefix dict has been built succesfully.


In [7]:
def get_chinese_words(file_path):
    with open(file_path, "r", encoding = "utf-8") as f:
        return [line.split()[0] for line in f.readlines()]

In [8]:
CH_DICT = set(get_chinese_words("chinese_words.txt"))

In [18]:
import re

unigram_freq, bigram_freq = {},{}
for i in range(len(words)-1):
    if words[i] not in CH_DICT and not re.search("[^\u4e00-\u9fa5]",words[i]): 
        if words[i] in unigram_freq: # 一阶计数
            unigram_freq[words[i]] += 1
        else:
            unigram_freq[words[i]] = 1
    bigram = words[i]+words[i+1]
    if bigram not in CH_DICT and not re.search("[^\u4e00-\u9fa5]",bigram): 
        if bigram in bigram_freq:
            bigram_freq[bigram] += 1
        else:
            bigram_freq[bigram] = 1

In [19]:
unigram_freq_sorted = sorted(unigram_freq.items(), key = lambda d: d[1],reverse = True)
bigram_freq_sorted = sorted(bigram_freq.items(), key = lambda d: d[1],reverse = True)

In [20]:
print("unigram:\n",unigram_freq_sorted)
print("bigram:\n",bigram_freq_sorted)

unigram:
 [('机是', 1), ('边距', 1), ('化项', 1), ('中有', 1)]
bigram:
 [('支持向量', 4), ('分类的', 3), ('向量机', 3), ('铰链损失', 2), ('损失函数', 2), ('正确分类', 2), ('向量机是', 1), ('机是一类', 1), ('一类按', 1), ('按监督', 1), ('监督学习', 1), ('学习方式', 1), ('方式对', 1), ('对数据', 1), ('数据进行', 1), ('进行二元', 1), ('二元分类', 1), ('的广义', 1), ('广义线性', 1), ('线性分类器', 1), ('其决策', 1), ('决策边界', 1), ('边界是', 1), ('是对', 1), ('对学习', 1), ('学习样本', 1), ('样本求解', 1), ('求解的', 1), ('的最大', 1), ('最大边距', 1), ('边距超平面', 1), ('机使用', 1), ('使用铰链', 1), ('函数计算', 1), ('计算经验', 1), ('经验风险', 1), ('风险并', 1), ('并在', 1), ('在求解', 1), ('求解系统', 1), ('系统中', 1), ('中加入', 1), ('加入了', 1), ('了正则', 1), ('正则化项', 1), ('化项以', 1), ('以优化结构', 1), ('优化结构风险', 1), ('是一个', 1), ('一个具有', 1), ('具有稀疏', 1), ('稀疏性', 1), ('性和', 1), ('和稳健性', 1), ('稳健性的', 1), ('的分类器', 1), ('函数的', 1), ('的思想', 1), ('思想就是', 1), ('就是让', 1), ('让那些', 1), ('那些未能', 1), ('未能正确', 1), ('的和', 1), ('和正确', 1), ('的之间', 1), ('之间的', 1), ('的距离', 1), ('距离要', 1), ('要足够', 1), ('足够的', 1), ('的远', 1), ('机可以', 1), ('可以通过', 1), ('通过核', 1), 

### 基于自由疑固度以及左右邻字熵的新词挖掘

- 自由疑固度：表示一个字串的凝固程度。
- 左邻字熵与右邻字熵：表示一个字串左右搭配的丰富性。

<center>$$pmi(x,y) = log{\frac{P(x,y)}{P(x)P(y)}}$$</center>

<center>$$entropy(w) = -P(x_i)logP(x_i)$$</center>

### 《红楼梦》一书中的新词挖掘实战

- 数据获取及预处理；词典获取。
- 将数据进行切分获取所有切分出的候选单词，并且统计词频信息、候选新词左右出现的字的信息。
- 根据第二步中统计的进行 pmi 值以及左右邻字熵的计算。
- 设定各指标的阈值，根据其值获取最终的新词结果。

In [1]:
# 读取数据
import re

def preprocess_data(file_path):
    texts = []
    with open(file_path, "r", encoding = "utf-8") as f:
        for text in f.readlines():
            text = re.sub("[^\u4e00-\u9fa5。？．，！：]","",text.strip()) 
            text_splited = re.split("[。？．，！：]", text) 
            texts += text_splited
    texts = [text for text in texts if text is not ""]
    return texts

In [2]:
texts = preprocess_data("hongloumeng.txt") # 处理数据《红楼梦》一书，按照基本的标点符号进行切分
texts

['红楼梦曹雪芹',
 '第一回甄士隐梦幻识通灵贾雨村风尘怀闺秀',
 '此开卷第一回也',
 '作者自云',
 '因曾历过一番梦幻之后',
 '故将真事隐去',
 '而借通灵之说',
 '撰此石头记一书也',
 '故曰甄士隐云云',
 '但书中所记何事何人',
 '自又云',
 '今风尘碌碌',
 '一事无成',
 '忽念及当日所有之女子',
 '一一细考较去',
 '觉其行止见识',
 '皆出于我之上',
 '何我堂堂须眉',
 '诚不若彼裙钗哉',
 '实愧则有余',
 '悔又无益之大无可如何之日也',
 '当此',
 '则自欲将已往所赖天恩祖德',
 '锦衣纨绔之时',
 '饫甘餍肥之日',
 '背父兄教育之恩',
 '负师友规谈之德',
 '以至今日一技无成',
 '半生潦倒之罪',
 '编述一集',
 '以告天下人',
 '我之罪固不免',
 '然闺阁中本自历历有人',
 '万不可因我之不肖',
 '自护己短',
 '一并使其泯灭也',
 '虽今日之茅椽蓬牖',
 '瓦灶绳床',
 '其晨夕风露',
 '阶柳庭花',
 '亦未有妨我之襟怀笔墨者',
 '虽我未学',
 '下笔无文',
 '又何妨用假语村言',
 '敷演出一段故事来',
 '亦可使闺阁昭传',
 '复可悦世之目',
 '破人愁闷',
 '不亦宜乎',
 '故曰贾雨村云云',
 '此回中凡用梦用幻等字',
 '是提醒阅者眼目',
 '亦是此书立意本旨',
 '列位看官',
 '你道此书从何而来',
 '说起根由虽近荒唐',
 '细按则深有趣味',
 '待在下将此来历注明',
 '方使阅者了然不惑',
 '原来女娲氏炼石补天之时',
 '于大荒山无稽崖练成高经十二丈',
 '方经二十四丈顽石三万六千五百零一块',
 '娲皇氏只用了三万六千五百块',
 '只单单剩了一块未用',
 '便弃在此山青埂峰下',
 '谁知此石自经煅炼之后',
 '灵性已通',
 '因见众石俱得补天',
 '独自己无材不堪入选',
 '遂自怨自叹',
 '日夜悲号惭愧',
 '一日',
 '正当嗟悼之际',
 '俄见一僧一道远远而来',
 '生得骨骼不凡',
 '丰神迥异',
 '说说笑笑来至峰下',
 '坐于石边高谈快论',
 '先是说些云山雾海神仙玄幻之事',
 '后便说到红尘中荣华富贵',
 '此石听了',
 '不

In [3]:
# 获取已有的中文词典
def get_chinese_words(file_path):
    with open(file_path, "r", encoding = "utf-8") as f:
        return [line.split()[0] for line in f.readlines()]

In [4]:
CH_DICT = set(get_chinese_words("chinese_words.txt"))

接下来需要对文本进行切分以及获取相关的频次信息，这里统一在一个函数中，主要逻辑如下：

- 对文本按照一定的长度范围进行切分，切分出所有成词的可能性，这里称之为字符串。
- 对于所有切分出的字符串进行过滤，长度大于等于 2 的词以及不是词典 CH_DICT 中的词作为候选新词。
- 获取所有切分出的字符串的频次信息（在后续计算中需要用到一些字符串的频次信息）、候选新词词频信息、候选新词左右出现的字的统计信息。

In [5]:
def get_candidate_wordsinfo(texts, max_word_len):
    # texts 表示输入的所有文本，max_word_len 表示最长的词长
    # 四个词典均以单词为 key，分别以词频、词频、左字集合、右字集合为 value
    words_freq, candidate_words_freq,candidate_words_left_characters, candidate_words_right_characters = {},{},{},{}
    WORD_NUM = 0 # 统计所有可能的字符串频次
    for text in texts: # 遍历每个文本
        # word_indexes 中存储了所有可能的词汇的切分下标 (i,j) ，i 表示词汇的起始下标，j 表示结束下标，注意这里有包括了所有的字
        # word_indexes 的生成需要两层循环，第一层循环，遍历所有可能的起始下标 i；第二层循环，在给定 i 的情况下，遍历所有可能的结束下标 j
        word_indexes = [(i,j) for i in range(len(text)) for j in range(i + 1, i + 1 + max_word_len)]
        WORD_NUM += len(word_indexes)
        for index in word_indexes: # 遍历所有词汇的下标
            word = text[index[0]:index[1]] # 获取单词
            # 更新所有切分出的字符串的频次信息
            if word in words_freq:
                words_freq[word] += 1
            else:
                words_freq[word] = 1
            if len(word) >= 2 and word not in CH_DICT: # 长度大于等于 2 的词以及不是词典中的词作为候选新词
                # 更新候选新词词频
                if word in candidate_words_freq:
                    candidate_words_freq[word] += 1
                else:
                    candidate_words_freq[word] = 1
                # 更新候选新词左字集合
                if index[0] != 0: # 当为文本中首个单词时无左字
                    if word in candidate_words_left_characters:
                        candidate_words_left_characters[word].append(text[index[0]-1])
                    else:
                        candidate_words_left_characters[word] = [text[index[0]-1]]
                # 更新候选新词右字集合
                if index[1] < len(text)-1: # 当为文本中末个单词时无右字
                    if word in candidate_words_right_characters:
                        candidate_words_right_characters[word].append(text[index[1]+1]) # 
                    else:
                        candidate_words_right_characters[word] = [text[index[1]+1]]
    return WORD_NUM, words_freq, candidate_words_freq, candidate_words_left_characters, candidate_words_right_characters

In [6]:
WORD_NUM, words_freq, candidate_words_freq, candidate_words_left_characters, candidate_words_right_characters = \
get_candidate_wordsinfo(texts = texts, max_word_len = 3) # 字符串最长为 3

在下一步中，计算 pmi 值以及左右邻字熵。

In [7]:
import math
# 计算候选单词的 pmi 值
def compute_pmi(words_freq,candidate_words):
    words_pmi = {}
    for word in candidate_words:
        # 首先，将某个候选单词按照不同的切分位置切分成两项，比如“电影院”可切分为“电”和“影院”以及“电影”和“院”
        bi_grams = [(word[0:i],word[i:]) for i in range(1,len(word))]
        # 对所有切分情况计算 pmi 值，取最大值作为当前候选词的最终 pmi 值
        # words_freq[bi_gram[0]]，words_freq[bi_gram[1]] 分别表示一个候选儿童村新词的前后两部分的出现频次
        words_pmi[word] = max(map(lambda bi_gram: math.log(\
        words_freq[word]/(words_freq[bi_gram[0]]*words_freq[bi_gram[1]]/WORD_NUM)),bi_grams))
        """
        通俗版本
        pmis = []
        for bi_gram in bigrams: # 遍历所有切分情况
            pmis.append(math.log(words_freq[word]/(words_freq[bi_gram[0]]*words_freq[bi_gram[1]]/WORD_NUM))) # 计算 pmi 值
        words_pmi[word] = max(pmis) # 取最大值
        """
    return words_pmi 

In [8]:
words_pmi = compute_pmi(words_freq,candidate_words_freq)

In [25]:
words_pmi

{'楼梦': 7.0587357419432335,
 '楼梦曹': 9.870716865893675,
 '梦曹': 6.286945434064175,
 '梦曹雪': 8.733466784520152,
 '曹雪': 7.509691352898036,
 '雪芹': 6.388917492427074,
 '一回甄': 3.804819724002924,
 '回甄': 2.906744525095319,
 '回甄士': 8.414218246100537,
 '甄士': 6.468308097045223,
 '士隐': 8.401639463893677,
 '士隐梦': 9.10736542666048,
 '隐梦': 4.044984962886565,
 '隐梦幻': 10.294531112670036,
 '梦幻识': 9.120158778120391,
 '幻识': 5.331686482410129,
 '幻识通': 10.294531112670036,
 '识通': 4.127713678084193,
 '识通灵': 9.014369169986555,
 '通灵': 6.624772700002879,
 '通灵贾': 9.0905583083441,
 '灵贾': 0.484457206162542,
 '灵贾雨': 8.745197124305642,
 '贾雨': 3.2110174340356203,
 '雨村': 8.3843424459642,
 '雨村风': 8.052049943745697,
 '村风尘': 9.324173159525605,
 '风尘怀': 8.255839230163756,
 '尘怀': 4.961074534737243,
 '尘怀闺': 10.582213185121818,
 '怀闺': 5.525967379773509,
 '怀闺秀': 10.428062505294559,
 '此开': 0.8340688441840427,
 '此开卷': 9.643943546528886,
 '开卷第': 8.824994041145938,
 '卷第': 4.59088753654868,
 '卷第一': 6.0196026135525225,
 '一回也': 3.8048197

In [9]:
from collections import Counter 
# 计算候选单词的邻字熵
def compute_entropy(candidate_words_characters):
    words_entropy = {}
    for word,characters in candidate_words_characters.items():
        character_freq = Counter(characters) # 统计邻字的出现分布
        # 根据出现分布计算邻字熵
        words_entropy[word] = sum(map(lambda x: - x/len(characters) * math.log(x/len(characters)) , character_freq.values())) 
    return words_entropy

In [10]:
words_left_entropy = compute_entropy(candidate_words_left_characters)
words_right_entropy = compute_entropy(candidate_words_right_characters)

In [20]:
# 根据各指标阈值获取最终的新词结果
def get_newwords(candidate_words_freq,words_pmi,words_left_entropy,words_right_entropy,\
                 words_freq_limit = 15, pmi_limit = 6, entropy_limit = 1):
    # 在每一项指标中根据阈值进行筛选
    candidate_words = [k for k, v in candidate_words_freq.items() if v >= words_freq_limit]    
    candidate_words_pmi = [k for k, v in words_pmi.items() if v >= pmi_limit]
    candidate_words_left_entropy = [k for k, v in words_left_entropy.items() if v >= entropy_limit]
    candidate_words_right_entropy = [k for k, v in words_right_entropy.items() if v >= entropy_limit]
    # 对筛选结果进行合并
    return list(set(candidate_words).intersection(candidate_words_pmi,candidate_words_left_entropy,candidate_words_right_entropy))

In [21]:
get_newwords(candidate_words_freq,words_pmi,words_left_entropy,words_right_entropy)

['在贾母',
 '到如今',
 '进园来',
 '凤姐也',
 '什么不',
 '的主意',
 '和尚道',
 '龄官',
 '佩凤',
 '如今这',
 '太太们',
 '士隐',
 '潇湘馆',
 '不理论',
 '告诉我',
 '王仁',
 '晴雯等',
 '秋桐',
 '袭人麝',
 '尤氏',
 '鸳鸯的',
 '忽然想',
 '命小丫',
 '我自己',
 '一把拉',
 '又嘱咐',
 '舅太爷',
 '黛玉又',
 '太太给',
 '忙站起',
 '自己也',
 '的衣服',
 '如今不',
 '的告诉',
 '爷那边',
 '和王夫',
 '贾政的',
 '也十分',
 '婆子们',
 '告诉贾',
 '什么话',
 '宝姑娘',
 '薛蝌',
 '小厮们',
 '妹妹的',
 '贾政等',
 '屋里的',
 '了薛姨',
 '的亲戚',
 '的姑娘',
 '日一早',
 '锄药',
 '媳妇们',
 '见这般',
 '原来是',
 '金桂',
 '打发人',
 '东西来',
 '怎么又',
 '你林妹',
 '位姑娘',
 '雨村道',
 '袭人姐',
 '这几年',
 '和凤姐',
 '见如此',
 '个地方',
 '贾琏的',
 '岂有不',
 '三姑娘',
 '有造化',
 '见邢夫',
 '个媳妇',
 '哥哥的',
 '咱们的',
 '姑娘来',
 '明白了',
 '和宝钗',
 '麝月等',
 '丫鬟们',
 '藕官',
 '你如今',
 '我母亲',
 '我告诉',
 '外书房',
 '丫头来',
 '衙门里',
 '邢岫',
 '袭人笑',
 '连忙起',
 '姑娘送',
 '见林黛',
 '太太叫',
 '的地方',
 '王夫',
 '大太太',
 '皆系',
 '在里间',
 '姑娘这',
 '连忙答',
 '见王夫',
 '太太的',
 '个老婆',
 '的光景',
 '手里拿',
 '请老爷',
 '姑娘们',
 '你妹妹',
 '李贵',
 '儿晚上',
 '在炕沿',
 '李纨',
 '焙茗',
 '秋纹',
 '咱们两',
 '自己不',
 '琏二爷',
 '打发我',
 '太太屋',
 '十两银',
 '李绮',
 '他素日',
 '厨房里',
 '说一句',
 '请大夫',
 '请二爷',
 '春燕',
 '四

In [32]:
# 改变参数
get_newwords(candidate_words_freq,words_pmi,words_left_entropy,words_right_entropy,
             words_freq_limit = 100, pmi_limit = 3,entropy_limit = 3)

['芳官',
 '赵姨',
 '坐着',
 '也没',
 '丫头们',
 '为什',
 '了宝玉',
 '有一个',
 '宝钗',
 '雨村',
 '尤氏',
 '告诉了',
 '周瑞家',
 '林姑',
 '也不知',
 '凤姐',
 '给你',
 '贾赦',
 '是谁',
 '听了',
 '一个人',
 '紫鹃',
 '叫我',
 '带着',
 '宝玉的',
 '袭人',
 '给他',
 '贾环',
 '给我',
 '叫他',
 '薛蝌',
 '贾蓉',
 '金桂',
 '妙玉',
 '打发人',
 '与他',
 '都是',
 '宝蟾',
 '和他',
 '也不敢',
 '二奶奶',
 '才是',
 '有什么',
 '邢夫',
 '薛蟠',
 '贾珍',
 '贾琏',
 '这几',
 '也不',
 '在那里',
 '薛姨',
 '贾母',
 '岂不',
 '见过',
 '有什',
 '王夫',
 '在这里',
 '便叫',
 '贾政',
 '贾芸',
 '林黛',
 '巧姐',
 '惜春',
 '不知道',
 '两个人',
 '拿着',
 '在那',
 '宝琴',
 '有几',
 '刘姥',
 '并不',
 '了一个',
 '的东西',
 '和你',
 '姑娘们',
 '的丫头',
 '在这',
 '李纨',
 '忘了',
 '不过是',
 '麝月',
 '雪雁',
 '秦钟',
 '黛玉',
 '莺儿']